# Introduction

## Motivation

## Data


Minsu

# (Adaptive) Kalman Filter

## Continuous State Space Model

## Connection to MVN-MVN Conjugacy

## Parameter Estimation

## Adaptivity


Wicak

# Challenges

## Markov Assumption

## Covariance Matrices

## Dynamic Systems


Minsu

# Model Fitting

## Forward Pass

## EM Smoothing

## PSIS-LFO-CV

## Constraints


Wicak

# Results

## Predictive Performance

## Latent State Estimates



# Criticism

## Computational Complexity

## Acceleration Term

Minsu

# Conclusion

Wicak

# References

__[PSIS](https://arxiv.org/pdf/1507.02646.pdf)__ Vehtari et al. (2019). Pareto Smoothed Importance Sampling
__[PSIS-LOO-CV](https://arxiv.org/pdf/1507.04544.pdf)__ Vehtari et al. (2016) Practical Bayesian model evaluation using leave-one-out cross-validation and WAIC
__[PSIS-LFO-CV](https://arxiv.org/pdf/1902.06281.pdf)__ Bürkner et al. (2019). Approximate leave-future-out cross-validation for Bayesian time series models

